In [1]:
import numpy as np 
import tensorflow as tf
import re
import time 

## Data Preprocessing

In [2]:
# Loading the movie Lines and conversations datasets
lines = open('data/movie_lines.txt', encoding='utf-8', errors='ignore').read().split('\n')
conversations = open('data/movie_conversations.txt', encoding='utf-8', errors='ignore').read().split('\n')
lines

['L1045 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ They do not!',
 'L1044 +++$+++ u2 +++$+++ m0 +++$+++ CAMERON +++$+++ They do to!',
 'L985 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ I hope so.',
 'L984 +++$+++ u2 +++$+++ m0 +++$+++ CAMERON +++$+++ She okay?',
 "L925 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ Let's go.",
 'L924 +++$+++ u2 +++$+++ m0 +++$+++ CAMERON +++$+++ Wow',
 "L872 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ Okay -- you're gonna need to learn how to lie.",
 'L871 +++$+++ u2 +++$+++ m0 +++$+++ CAMERON +++$+++ No',
 'L870 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ I\'m kidding.  You know how sometimes you just become this "persona"?  And you don\'t know how to quit?',
 'L869 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ Like my fear of wearing pastels?',
 'L868 +++$+++ u2 +++$+++ m0 +++$+++ CAMERON +++$+++ The "real you".',
 'L867 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ What good stuff?',
 "L866 +++$+++ u2 +++$+++ m0 +++$+++ CAMERON +++$+++ I figured yo

In [3]:
conversations

["u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L194', 'L195', 'L196', 'L197']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L198', 'L199']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L200', 'L201', 'L202', 'L203']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L204', 'L205', 'L206']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L207', 'L208']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L271', 'L272', 'L273', 'L274', 'L275']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L276', 'L277']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L280', 'L281']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L363', 'L364']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L365', 'L366']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L367', 'L368']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L401', 'L402', 'L403']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L404', 'L405', 'L406', 'L407']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L575', 'L576']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L577', 'L578']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L662', 'L663']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L693', 'L69

In [4]:
# Arranging the lines data into a dictionary (Question and Answer)

id2line = {}
for line in lines:
    _line = line.split(' +++$+++ ')
    if len(_line) == 5:
        id2line[_line[0]] = _line[4]

id2line

{'L1045': 'They do not!',
 'L1044': 'They do to!',
 'L985': 'I hope so.',
 'L984': 'She okay?',
 'L925': "Let's go.",
 'L924': 'Wow',
 'L872': "Okay -- you're gonna need to learn how to lie.",
 'L871': 'No',
 'L870': 'I\'m kidding.  You know how sometimes you just become this "persona"?  And you don\'t know how to quit?',
 'L869': 'Like my fear of wearing pastels?',
 'L868': 'The "real you".',
 'L867': 'What good stuff?',
 'L866': "I figured you'd get to the good stuff eventually.",
 'L865': 'Thank God!  If I had to hear one more story about your coiffure...',
 'L864': "Me.  This endless ...blonde babble. I'm like, boring myself.",
 'L863': 'What crap?',
 'L862': 'do you listen to this crap?',
 'L861': 'No...',
 'L860': 'Then Guillermo says, "If you go any lighter, you\'re gonna look like an extra on 90210."',
 'L699': 'You always been this selfish?',
 'L698': 'But',
 'L697': "Then that's all you had to say.",
 'L696': 'Well, no...',
 'L695': "You never wanted to go out with 'me, did y

In [5]:
# List of conversations from all the conversations

conversations_ids = []
for conversation in conversations[:-1]:
    _conversation = conversation.split(' +++$+++ ')[-1][1:-1].replace("'", "").replace(" ", "")
    conversations_ids.append(_conversation.split(','))
    
    
conversations_ids

[['L194', 'L195', 'L196', 'L197'],
 ['L198', 'L199'],
 ['L200', 'L201', 'L202', 'L203'],
 ['L204', 'L205', 'L206'],
 ['L207', 'L208'],
 ['L271', 'L272', 'L273', 'L274', 'L275'],
 ['L276', 'L277'],
 ['L280', 'L281'],
 ['L363', 'L364'],
 ['L365', 'L366'],
 ['L367', 'L368'],
 ['L401', 'L402', 'L403'],
 ['L404', 'L405', 'L406', 'L407'],
 ['L575', 'L576'],
 ['L577', 'L578'],
 ['L662', 'L663'],
 ['L693', 'L694', 'L695'],
 ['L696', 'L697', 'L698', 'L699'],
 ['L860', 'L861'],
 ['L862', 'L863', 'L864', 'L865'],
 ['L866', 'L867', 'L868', 'L869'],
 ['L870', 'L871', 'L872'],
 ['L924', 'L925'],
 ['L984', 'L985'],
 ['L1044', 'L1045'],
 ['L49', 'L50', 'L51'],
 ['L571', 'L572', 'L573'],
 ['L579', 'L580'],
 ['L595', 'L596', 'L597'],
 ['L598', 'L599', 'L600'],
 ['L659', 'L660'],
 ['L952', 'L953'],
 ['L394', 'L395'],
 ['L396', 'L397'],
 ['L589', 'L590', 'L591'],
 ['L592', 'L593'],
 ['L756', 'L757', 'L758'],
 ['L759', 'L760'],
 ['L164', 'L165'],
 ['L319', 'L320'],
 ['L441', 'L442', 'L443', 'L444', 'L445']

In [6]:
# Processing the data to get all data into the format of Questions and Answers

questions = []
answers = []

for conversation in conversations_ids:
    for i in range(len(conversation) - 1):
        questions.append(id2line[conversation[i]])
        answers.append(id2line[conversation[i+1]])



In [7]:
print(questions[0:10])
print(answers[0:10])

['Can we make this quick?  Roxanne Korrine and Andrew Barrett are having an incredibly horrendous public break- up on the quad.  Again.', "Well, I thought we'd start with pronunciation, if that's okay with you.", 'Not the hacking and gagging and spitting part.  Please.', "You're asking me out.  That's so cute. What's your name again?", "No, no, it's my fault -- we didn't have a proper introduction ---", 'Cameron.', "The thing is, Cameron -- I'm at the mercy of a particularly hideous breed of loser.  My sister.  I can't date until she does.", 'Why?', 'Unsolved mystery.  She used to be really popular when she started high school, then it was just like she got sick of it or something.', 'Gosh, if only we could find Kat a boyfriend...']
["Well, I thought we'd start with pronunciation, if that's okay with you.", 'Not the hacking and gagging and spitting part.  Please.', "Okay... then how 'bout we try out some French cuisine.  Saturday?  Night?", 'Forget it.', 'Cameron.', "The thing is, Came

In [8]:
# Data Clearning
def clean_text(text):
    # Lower case all the data
    text = text.lower()
    
    # Removing the "'"
    text = re.sub(r"i'm", "i am", text)
    text = re.sub(r"he's", "he is", text)
    text = re.sub(r"she's", "she is", text)
    text = re.sub(r"that's", "that is", text)
    text = re.sub(r"what's", "what is", text)
    text = re.sub(r"where's", "where is", text)
    text = re.sub(r"\'ll", "will", text)
    text = re.sub(r"\'ve", "have", text)
    text = re.sub(r"\'re", "are", text)
    text = re.sub(r"\'d", "would", text)
    text = re.sub(r"won't", "will not", text)
    text = re.sub(r"can't", "cannot", text)
    
    # Removing symbols which are not needed
    text = re.sub(r"[-()\"#/@;:<>{}+=~|.?,]", "", text)
    return text

In [9]:
# Clearning the Questions
clean_questions = []
for question in questions:
    clean_questions.append(clean_text(question))

In [10]:
# Clearning the Answers
clean_answers = [clean_text(answer)  for answer in answers]
clean_answers[0:10]

['well i thought wewould start with pronunciation if that is okay with you',
 'not the hacking and gagging and spitting part  please',
 "okay then how 'bout we try out some french cuisine  saturday  night",
 'forget it',
 'cameron',
 'the thing is cameron  i am at the mercy of a particularly hideous breed of loser  my sister  i cannot date until she does',
 'seems like she could get a date easy enough',
 'unsolved mystery  she used to be really popular when she started high school then it was just like she got sick of it or something',
 'that is a shame',
 'let me see what i can do']

In [11]:
clean_questions[0:10]

['can we make this quick  roxanne korrine and andrew barrett are having an incredibly horrendous public break up on the quad  again',
 'well i thought wewould start with pronunciation if that is okay with you',
 'not the hacking and gagging and spitting part  please',
 'youare asking me out  that is so cute what is your name again',
 "no no it's my fault  we didn't have a proper introduction ",
 'cameron',
 'the thing is cameron  i am at the mercy of a particularly hideous breed of loser  my sister  i cannot date until she does',
 'why',
 'unsolved mystery  she used to be really popular when she started high school then it was just like she got sick of it or something',
 'gosh if only we could find kat a boyfriend']

In [12]:
# Removing the non-frequently occuring words
# Appraoch is to create a dictionary that maps each word to its occurances

word2count = {}
for question in clean_questions:
    for word in question.split():
        if word not in word2count:
            word2count[word] = 1
        else:
            word2count[word] += 1
            
for answer in clean_answers:
    for word in answer.split():
        if word not in word2count:
            word2count[word] = 1
        else:
            word2count[word] += 1


In [13]:
# We are tokenizing and also ensuring that only words above of certain frequence of occurance are insert into the new dictionaries
# Creating two new dictionaries that map the questions words and the answers words to a unique integer
# we will chose the threshold of filtering the words
threshold = 20
questionswords2int = {}
word_number = 0
for word, count in word2count.items():
    if count >= threshold:
        questionswords2int[word] = word_number
        word_number += 1
        
threshold = 20
answerswords2int = {}
word_number = 0
for word, count in word2count.items():
    if count >= threshold:
        answerswords2int[word] = word_number
        word_number += 1


In [14]:
questionswords2int

{'can': 0,
 'we': 1,
 'make': 2,
 'this': 3,
 'quick': 4,
 'and': 5,
 'andrew': 6,
 'are': 7,
 'having': 8,
 'an': 9,
 'incredibly': 10,
 'public': 11,
 'break': 12,
 'up': 13,
 'on': 14,
 'the': 15,
 'again': 16,
 'well': 17,
 'i': 18,
 'thought': 19,
 'wewould': 20,
 'start': 21,
 'with': 22,
 'if': 23,
 'that': 24,
 'is': 25,
 'okay': 26,
 'you': 27,
 'not': 28,
 'part': 29,
 'please': 30,
 'youare': 31,
 'asking': 32,
 'me': 33,
 'out': 34,
 'so': 35,
 'cute': 36,
 'what': 37,
 'your': 38,
 'name': 39,
 'no': 40,
 "it's": 41,
 'my': 42,
 'fault': 43,
 "didn't": 44,
 'have': 45,
 'a': 46,
 'proper': 47,
 'cameron': 48,
 'thing': 49,
 'am': 50,
 'at': 51,
 'mercy': 52,
 'of': 53,
 'particularly': 54,
 'breed': 55,
 'loser': 56,
 'sister': 57,
 'cannot': 58,
 'date': 59,
 'until': 60,
 'she': 61,
 'does': 62,
 'why': 63,
 'mystery': 64,
 'used': 65,
 'to': 66,
 'be': 67,
 'really': 68,
 'popular': 69,
 'when': 70,
 'started': 71,
 'high': 72,
 'school': 73,
 'then': 74,
 'it': 75,
 'w

In [15]:
# Adding the last tokens to these two dictionaries

tokens = ['<PAD>', '<EOS>', '<OUT>', '<SOS>']
for token in tokens:
    questionswords2int[token] = len(questionswords2int) + 1
    answerswords2int[token] = len(answerswords2int) + 1

In [16]:
# Creating the inverse dictionary of the answerswords2int dictionary
answersints2word = {w_i: w for w, w_i in answerswords2int.items()}
answersints2word

{0: 'can',
 1: 'we',
 2: 'make',
 3: 'this',
 4: 'quick',
 5: 'and',
 6: 'andrew',
 7: 'are',
 8: 'having',
 9: 'an',
 10: 'incredibly',
 11: 'public',
 12: 'break',
 13: 'up',
 14: 'on',
 15: 'the',
 16: 'again',
 17: 'well',
 18: 'i',
 19: 'thought',
 20: 'wewould',
 21: 'start',
 22: 'with',
 23: 'if',
 24: 'that',
 25: 'is',
 26: 'okay',
 27: 'you',
 28: 'not',
 29: 'part',
 30: 'please',
 31: 'youare',
 32: 'asking',
 33: 'me',
 34: 'out',
 35: 'so',
 36: 'cute',
 37: 'what',
 38: 'your',
 39: 'name',
 40: 'no',
 41: "it's",
 42: 'my',
 43: 'fault',
 44: "didn't",
 45: 'have',
 46: 'a',
 47: 'proper',
 48: 'cameron',
 49: 'thing',
 50: 'am',
 51: 'at',
 52: 'mercy',
 53: 'of',
 54: 'particularly',
 55: 'breed',
 56: 'loser',
 57: 'sister',
 58: 'cannot',
 59: 'date',
 60: 'until',
 61: 'she',
 62: 'does',
 63: 'why',
 64: 'mystery',
 65: 'used',
 66: 'to',
 67: 'be',
 68: 'really',
 69: 'popular',
 70: 'when',
 71: 'started',
 72: 'high',
 73: 'school',
 74: 'then',
 75: 'it',
 76

In [17]:
# Adding the End of String token to the end of every answer
for i in range(len(clean_answers)):
    clean_answers[i] += ' <EOS>'

clean_answers[0:10]

['well i thought wewould start with pronunciation if that is okay with you <EOS>',
 'not the hacking and gagging and spitting part  please <EOS>',
 "okay then how 'bout we try out some french cuisine  saturday  night <EOS>",
 'forget it <EOS>',
 'cameron <EOS>',
 'the thing is cameron  i am at the mercy of a particularly hideous breed of loser  my sister  i cannot date until she does <EOS>',
 'seems like she could get a date easy enough <EOS>',
 'unsolved mystery  she used to be really popular when she started high school then it was just like she got sick of it or something <EOS>',
 'that is a shame <EOS>',
 'let me see what i can do <EOS>']

In [18]:
# Translating all the question and answers into integers
# replace all the words that were filtered out by <OUT>

questions_into_int = []
for question in clean_questions:
    question_ints = []
    for word in question.split():
        if word not in questionswords2int:
            question_ints.append(questionswords2int['<OUT>'])
        else:
            question_ints.append(questionswords2int[word])
    questions_into_int.append(question_ints)

# Repeating this for answers too
answers_into_int = []
for answer in clean_answers:
    answer_ints = []
    for word in answer.split():
        if word not in answerswords2int:
            answer_ints.append(answerswords2int['<OUT>'])
        else:
            answer_ints.append(answerswords2int[word])
    answers_into_int.append(question_ints)


In [19]:
questions_into_int[0:10]

[[0,
  1,
  2,
  3,
  4,
  8867,
  8867,
  5,
  6,
  8867,
  7,
  8,
  9,
  10,
  8867,
  11,
  12,
  13,
  14,
  15,
  8867,
  16],
 [17, 18, 19, 20, 21, 22, 8867, 23, 24, 25, 26, 22, 27],
 [28, 15, 8867, 5, 8867, 5, 8867, 29, 30],
 [31, 32, 33, 34, 24, 25, 35, 36, 37, 25, 38, 39, 16],
 [40, 40, 41, 42, 43, 1, 44, 45, 46, 47, 8867],
 [48],
 [15,
  49,
  25,
  48,
  18,
  50,
  51,
  15,
  52,
  53,
  46,
  54,
  8867,
  55,
  53,
  56,
  42,
  57,
  18,
  58,
  59,
  60,
  61,
  62],
 [63],
 [8867,
  64,
  61,
  65,
  66,
  67,
  68,
  69,
  70,
  61,
  71,
  72,
  73,
  74,
  75,
  76,
  77,
  78,
  61,
  79,
  80,
  53,
  75,
  81,
  82],
 [83, 23, 84, 1, 85, 86, 87, 46, 88]]

In [20]:
answers_into_int[0:10]

[[5,
  18,
  1694,
  27,
  27,
  129,
  28,
  112,
  1656,
  287,
  67,
  5323,
  94,
  38,
  1458,
  1632,
  37,
  45,
  38,
  6116,
  602],
 [5,
  18,
  1694,
  27,
  27,
  129,
  28,
  112,
  1656,
  287,
  67,
  5323,
  94,
  38,
  1458,
  1632,
  37,
  45,
  38,
  6116,
  602],
 [5,
  18,
  1694,
  27,
  27,
  129,
  28,
  112,
  1656,
  287,
  67,
  5323,
  94,
  38,
  1458,
  1632,
  37,
  45,
  38,
  6116,
  602],
 [5,
  18,
  1694,
  27,
  27,
  129,
  28,
  112,
  1656,
  287,
  67,
  5323,
  94,
  38,
  1458,
  1632,
  37,
  45,
  38,
  6116,
  602],
 [5,
  18,
  1694,
  27,
  27,
  129,
  28,
  112,
  1656,
  287,
  67,
  5323,
  94,
  38,
  1458,
  1632,
  37,
  45,
  38,
  6116,
  602],
 [5,
  18,
  1694,
  27,
  27,
  129,
  28,
  112,
  1656,
  287,
  67,
  5323,
  94,
  38,
  1458,
  1632,
  37,
  45,
  38,
  6116,
  602],
 [5,
  18,
  1694,
  27,
  27,
  129,
  28,
  112,
  1656,
  287,
  67,
  5323,
  94,
  38,
  1458,
  1632,
  37,
  45,
  38,
  6116,
  602],
 [5,
 

In [21]:
# Sorting questions and answers by the length of the questions
# This ensures the performance of the model is faster

sorted_clean_questions = []
sorted_clean_answers = []

# ensure that the questions length is around 25 words, this will be fine tunned after the model is created
for length in range(1, 25+1):
    for i in enumerate(questions_into_int):
        if len(i[1]) == length:
            sorted_clean_questions.append(questions_into_int[i[0]])
            sorted_clean_answers.append(answers_into_int[i[0]])

In [22]:
sorted_clean_questions

[[48],
 [63],
 [124],
 [148],
 [136],
 [40],
 [177],
 [40],
 [184],
 [185],
 [225],
 [37],
 [63],
 [136],
 [63],
 [111],
 [385],
 [133],
 [304],
 [214],
 [37],
 [226],
 [184],
 [26],
 [63],
 [184],
 [8867],
 [471],
 [255],
 [184],
 [195],
 [8867],
 [8867],
 [691],
 [98],
 [8867],
 [8867],
 [37],
 [40],
 [37],
 [8867],
 [148],
 [91],
 [37],
 [794],
 [8867],
 [659],
 [40],
 [40],
 [964],
 [8867],
 [1146],
 [40],
 [236],
 [70],
 [214],
 [40],
 [143],
 [1294],
 [214],
 [1138],
 [1138],
 [1138],
 [1138],
 [214],
 [349],
 [150],
 [26],
 [93],
 [691],
 [37],
 [1166],
 [1288],
 [8867],
 [8867],
 [1539],
 [214],
 [1580],
 [37],
 [37],
 [214],
 [1629],
 [1629],
 [1629],
 [1629],
 [1629],
 [1629],
 [26],
 [691],
 [68],
 [691],
 [236],
 [98],
 [1659],
 [1629],
 [1629],
 [1629],
 [8867],
 [1629],
 [1629],
 [68],
 [1700],
 [696],
 [1820],
 [214],
 [39],
 [214],
 [1250],
 [214],
 [1377],
 [214],
 [17],
 [124],
 [37],
 [63],
 [1869],
 [1802],
 [214],
 [1877],
 [214],
 [214],
 [1852],
 [226],
 [1174],


## Bulding the Seq2Seq Model using TF

In [24]:
# Creating inputs and target tensors

def model_inputs():
    inputs = tf.placeholder(tf.int32, [None, None], name='input')
    targets = tf.placeholder(tf.int32, [None, None], name='target')
    lr = tf.placeholder(tf.float32, name='learning_rate')
    keep_prob = tf.placeholder(tf.float32, name='keep_prob')
    return inputs, targets, lr, keep_prob

In [25]:
# Creating the batches for targets and also add SOS token in the begining
# Take the last token and add it in the beginning

def preprocess_targets(targets, word2int, batch_size):
    left_side = tf.fill([batch_size, 1], word2int['<SOS>'])
    right_side = tf.strided_slice(targets, [0,0], [batch_size, -1], [1, 1])
    preprocessed_targets = tf.concat([left_side, right_side], axis=1)
    return preprocessed_targets

In [26]:
# Creating the Encoder RNN Layer

def encoder_rnn_layer(rnn_inputs, rnn_size, num_layers, keep_prob, sequence_length):
    lstm = tf.contrib.rnn.BasicLSTMCell(rnn_size)
    lstm_dropout = tf.contrib.rnn.DropoutWrapper(lstm, input_keep_prob=keep_prob)
    encoder_cell = tf.contrib.rnn.MultiRNNCell([lstm_dropout] * num_layers)
    encoder_output, encoder_state = tf.nn.bidirectional_dynamic_rnn(cell_fw = encoder_cell,
                                                      cell_bw = encoder_cell,
                                                      sequence_length = sequence_length,
                                                      inputs = rnn_inputs, 
                                                      dtype = tf.float32)
    return encoder_state

In [27]:
# Decoding the training set

def decode_training_set(encoder_state, decoder_cell, decoder_embedded_input, sequence_length, decoding_scope, 
                       output_function, keep_prob, batch_size):
    attention_states = tf.zeros([batch_size, 1, decoder_cell.output_size])
    attention_keys, attention_values, attention_score, attention_construct_function = tf.contrib.seq2seq.prepare_attention(attention_states, 
                                                                                                                          attention_option='bahdanau',
                                                                                                                          num_units = decoder_cell.output_size)
    training_decoder_function = tf.contrib.seq2seq.attention_decoder_fn_train(encoder_state[0], 
                                                                             attention_keys,
                                                                             attention_values,
                                                                             attention_score_function,
                                                                             attention_construct_function,
                                                                             name = "attn_dec_train")
    decoder_output, decoder_final_state, decoder_final_context_state = tf.contrib.seq2seq.dynamic_rnn_decoder(decoder_cell,
                                                                                                             training_decoder_function,
                                                                                                             decoder_embedded_input,
                                                                                                             sequence_length,
                                                                                                             scope = decoding_scope)
    decoder_output_dropout = tf.nn.dropout(decoder_output, keep_prob)
    return output_function(decoder_output_dropout)

In [ ]:
# Decoding the test/validation set
def decode_test_set(encoder_state, decoder_cell, decoder_embedded_input, sos_id, eos_id, maximum_length, num_words, 
                    sequence_length, decoding_scope, 
                       output_function, keep_prob, batch_size):
    attention_states = tf.zeros([batch_size, 1, decoder_cell.output_size])
    attention_keys, attention_values, attention_score, attention_construct_function = tf.contrib.seq2seq.prepare_attention(attention_states, 
                                                                                                                          attention_option='bahdanau',
                                                                                                                          num_units = decoder_cell.output_size)
    training_decoder_function = tf.contrib.seq2seq.attention_decoder_fn_(encoder_state[0], 
                                                                             attention_keys,
                                                                             attention_values,
                                                                             attention_score_function,
                                                                             attention_construct_function,
                                                                             name = "attn_dec_train")
    decoder_output, decoder_final_state, decoder_final_context_state = tf.contrib.seq2seq.dynamic_rnn_decoder(decoder_cell,
                                                                                                             training_decoder_function,
                                                                                                             decoder_embedded_input,
                                                                                                             sequence_length,
                                                                                                             scope = decoding_scope)
    decoder_output_dropout = tf.nn.dropout(decoder_output, keep_prob)
    return output_function(decoder_output_dropout)